In [ ]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pandas as pd
import json
import tqdm
import matplotlib.pyplot as plt

## 1. Data Generation

In [ ]:
# 2D functions used to compute targets
def sin_sqrt(X):
    return np.sin(np.sqrt(X[:, 0]**2 + X[:, 1]**2))

def booth(X):
    X1, X2 = X[:, 0], X[:, 1]
    return ((X1 + 2*X2 - 7)**2 + (2*X1 + X2 - 5)**2)

def goldstein_price(X):
    X1, X2 = X[:, 0], X[:, 1]
    return ((1+(X1 + X2 + 1)**2*(19-14*X1+3*X1**2-14*X2+6*X1*X2+3*X2**2))*(30+(2*X1-3*X2)**2*(18-32*X1+12*X1**2+48*X2-36*X1*X2+27*X2**2)))

def beale(X):
    X1, X2 = X[:, 0], X[:, 1]
    return (1.5 - X1 + X1*X2)**2 + (2.25 - X1 + X1*X2**2)**2 + (2.625 - X1 + X1*X2**3)**2

# function to generate 2D input data
def generate_X(n_samples, ranges=[(0, 1), (0, 1)], random_state=None):
    dim = len(ranges)
    assert dim > 0
    rng = np.random.default_rng(random_state)
    X = np.zeros((n_samples, dim))
    for i, r in enumerate(ranges):
        low, high = r
        X[:, i] = rng.uniform(low, high, n_samples)
    return X

In [ ]:
# generate datasets
seed = 42
n_samples = 10000
features = dict(
    sin_sqrt=generate_X(n_samples, ranges=[(-5, 5), (-5, 5)], random_state=seed),
    booth=generate_X(n_samples, ranges=[(-10, 10), (-10, 10)], random_state=seed),
    goldstein_price=generate_X(n_samples, ranges=[(-2, 2), (-2, 2)], random_state=seed),
    beale=generate_X(n_samples, ranges=[(-4.5, 4.5), (-4.5, 4.5)], random_state=seed)
)

targets = dict(
    sin_sqrt=sin_sqrt(features['sin_sqrt']),
    booth=booth(features['booth']),
    goldstein_price=goldstein_price(features['goldstein_price']),
    beale=beale(features['beale'])
)

In [ ]:
from matplotlib import colors
from scipy.interpolate import griddata

fn_map = dict(
    goldstein_price="Goldstein Price",
    booth="Booth",
    beale="Beale",
    sin_sqrt="SSR"
)

def plot_as_heatmap(ax, fig, x, y, z, n_points=100, cmap="viridis", norm="symlog", interpolation="cubic"):
    x_min, x_max = min(x), max(x)
    y_min, y_max = min(y), max(y)
    X, Y = np.meshgrid(np.linspace(x_min, x_max, n_points), np.linspace(y_min, y_max, n_points))
    Z = griddata((x, y), z, (X, Y), method=interpolation)
    pcm = ax.imshow(Z, extent=[x_min, x_max, y_min, y_max], origin="lower", cmap=cmap, norm=norm)
    return pcm

n_fns = len(targets)
n_rows = 2
height = 5
fig = plt.figure(figsize=(height*n_rows, height*n_rows))
for idx, fn in enumerate(features):
    ax = fig.add_subplot(n_rows, n_fns//n_rows, idx+1)
    pcm = plot_as_heatmap(ax, fig, features[fn][:, 0], features[fn][:, 1], targets[fn], n_points=1000)
    ax.set_title(fn_map[fn])
plt.show()

## 2. Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

train = {}
test = {}
scalers = {}
for k in features:
    X_train, X_test, y_train, y_test = train_test_split(features[k], targets[k], test_size=0.2)
    train[k] = (X_train, y_train)
    test[k] = (X_test, y_test)
    scalers[k] = MinMaxScaler((-1, 1))
    scalers[k].fit(X_train)
scalers

## 3. Training

### 3.1 K-Fold Cross-Validation Comparison

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, cross_validate
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from mas import context
from mas import head


# loading best params for benchmark algorithms
with open("best_params.json", "r") as f:
    best_params = json.load(f)

best_params["mas"] = dict(
    sin_sqrt=dict(
        R=np.full(2, 0.1),
        alpha=0.4,
        bad_th=0.2,
        imprecise_th=0.01,
        memory_length=20,
        min_vol=0.
    ),
    booth=dict(
        R=np.full(2, 0.35),
        alpha=0.1,
        bad_th=0.2,
        imprecise_th=0.15,
        memory_length=50,
        min_vol=0.
    ),
    goldstein_price=dict(
        R=np.full(2, 0.1),
        alpha=0.1,
        bad_th=0.2,
        imprecise_th=0.01,
        memory_length=200,
        min_vol=0.
    ),
    beale=dict(
        R=np.full(2, 0.05),
        alpha=0.2,
        bad_th=0.3,
        imprecise_th=0.1,
        memory_length=20,
        min_vol=0.
    ),
)

# initialize models from best params:
models_cls = dict(
    xgboost=XGBRegressor,
    lightbgm=LGBMRegressor,
    random_forest=RandomForestRegressor,
    decision_tree=DecisionTreeRegressor,
    linear_regression=LinearRegression,
    gradient_boosting=GradientBoostingRegressor,
    svr=SVR,
    mas=head.FastHeadAgent
)
models = {}
for k in best_params:
    models_by_fn = {}
    algo = models_cls[k]
    for fn, params in best_params[k].items():
        models_by_fn[fn] = algo(**params)
    models[k] = models_by_fn
models.keys()

In [ ]:
# evaluate cross validation score
cross_val_results = {}
scoring = {
    'r2': make_scorer(r2_score),
    'mse': make_scorer(mean_squared_error),
    'mae': make_scorer(mean_absolute_error)
}
pbar = tqdm.tqdm(models)
for m in pbar:
    res = {}
    for fn, model in models[m].items():
        pbar.set_description(f"[{m} - {fn}]")
        X_train, y_train = train[fn]
        score = cross_validate(model, scalers[fn].transform(X_train), y_train, cv=5, scoring=scoring, return_estimator=True, n_jobs=-1)
        res[fn] = score
    cross_val_results[m] = res

In [ ]:
from pprint import pprint

scores = {k: v for k,v in cross_val_results.items()}
scores = {k: {fn: {metric: values.mean() for metric, values in m.items() if metric.startswith("test_")} for fn, m in v.items()} for k,v in scores.items()}
results_df = pd.DataFrame.from_dict({(outerKey, innerKey): values for outerKey, innerDict in scores.items() for innerKey, values in innerDict.items()}, orient='index')
results_df = results_df.swaplevel().sort_index()
results_df.to_excel("results.xlsx")

### 3.2 Comparing predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# evaluate on test data
results = dict()

for fn in test:
    X_test, y_test = test[fn]
    res = {}
    for m in models:
        model = models[m][fn]
        y_pred = model.predict(scalers[fn].transform(X_test))
        mse = mean_squared_error(y_pred, y_test)
        mae = mean_absolute_error(y_pred, y_test)
        r2 = r2_score(y_pred, y_test)
        res[m] = dict(
            mse=mse, 
            mae=mae,
            r2=r2,
        )
    results[fn] = res

results_df = pd.DataFrame.from_dict({(outerKey, innerKey): values for outerKey, innerDict in results.items() for innerKey, values in innerDict.items()}, orient='index')
results_df

In [ ]:
ms = ["mas", "xgboost", "lightbgm"]
n_models = len(ms)
n_fns = len(test)
height = 6
width = 5
n_cols = n_models + 1
n_rows = n_fns

fig = plt.figure(figsize=(height * n_cols, width * n_rows), layout="constrained")
plt_idx = 0
for fn in test:
    X_test, y_test = test[fn]
    v_min, v_max = np.round(y_test.min()), np.round(y_test.max())
    
    for m in ms:
        plt_idx += 1
        y_pred = models[m][fn].predict(scalers[fn].transform(X_test))
        ax = fig.add_subplot(n_cols, n_rows, plt_idx)
        pcm_ = plot_as_heatmap(ax, fig, X_test[:, 0], X_test[:, 1], y_pred, n_points=1000, interpolation="linear")
        pcm_.set_clim(vmin=v_min, vmax=v_max)
        ax.set_title(f"prediction {m}")
    fig.colorbar(pcm_)
    
    plt_idx += 1